In [ ]:
!pip install jupyterlab_widgets

In [ ]:
!jupyter labextension install @jupyter-widgets/jupyterlab-manager@^2.0.0

In [ ]:
!jupyter lab build

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

In [ ]:
tags_df = pd.read_csv('tags.csv')      
movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')
print(tags_df.head())
print(movies_df.head())
print(ratings_df.head())

In [ ]:
movies_df['year'] = movies_df['title'].str.extract(r'\((\d{4})\)').astype(float)

In [ ]:
#merge tag and movie tables
tags_movies_df = pd.merge(tags_df, movies_df, on='movieId')

In [ ]:
#convert tag to lowercase for uniformity
tags_movies_df['tag']=tags_movies_df['tag'].str.lower()
print(tags_movies_df[['movieId', 'title', 'tag']].head())

In [ ]:
#define mood category
mood_keywords = {
    "happy": ["funny", "feel-good", "uplifting", "comedy"],
    "sad": ["tearjerker", "emotional", "tragic", "cry"],
    "romantic": ["romantic", "love", "date night"],
    "thrilling": ["thriller", "suspense", "intense", "edge of seat"],
    "exciting": ["action", "adventure", "fast-paced"],
    "scary": ["horror", "scary", "spooky", "creepy"],
    "inspirational": ["inspirational", "motivational", "biography", "life lesson"]}

In [ ]:
#rating summary
ratings_summary_df = ratings_df.groupby('movieId')['rating'].agg(['mean', 'count']).reset_index()
ratings_summary_df.columns = ['movieId', 'avg_rating', 'rating_count']

In [ ]:
#recommendation function
def get_movies_by_mood_with_rating (mood, top=10, min_rating = 100):
    keywords = mood_keywords.get(mood.lower())
    if not keywords:
        return pd.Dataframe() #return empty
    filtered = tags_movies_df[tags_movies_df['tag'].isin(keywords)]
    filtered = filtered.merge(ratings_summary_df, on = 'movieId', how='left')
    result = filtered[['movieId','title','tag','avg_rating','rating_count']].drop_duplicates()
    result = result[result['rating_count'] >= min_rating]
    result = result.sort_values(by='avg_rating', ascending=False).reset_index(drop=True)
    return result.head(top)

In [ ]:
#create a dropdown menu
mood_dropdown = widgets.Dropdown(
    options=[""] + list(mood_keywords.keys()),  # Add blank option
    description='Select Mood:',
    value=''  # default blank
)
#button to trigger recommendation
recommend_button = widgets.Button(description = "Recommend Movies")
output = widgets.Output()

In [52]:
#define on button click

def on_button_clicked(b):
    with output:
        clear_output()
        select_mood = mood_dropdown.value
        if not select_mood:
            print("⚠️ Please select a mood from the dropdown.")
        else:
            mood_movies = get_movies_by_mood_with_rating(select_mood)
            if isinstance(mood_movies, str):
                print(mood_movies)
            else:
                display(mood_movies.head(10))  # Show top 10 matches

recommend_button.on_click(on_button_clicked)

# Show dropdown and button
display(mood_dropdown, recommend_button, output)

Dropdown(description='Select Mood:', index=2, options=('', 'happy', 'sad', 'romantic', 'thrilling', 'exciting'…

Button(description='Recommend Movies', style=ButtonStyle())

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': "   movieId                            …